[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/XPretrain-colab/blob/main/test.ipynb)

In [ ]:
%cd /content

!pip install datasets yt_dlp scenedetect[opencv]

import datasets, json
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

dataset = datasets.load_dataset("camenduru/hdvila_test", split='train[:1%]')
# dataset = datasets.load_dataset("camenduru/microsoft-XPretrain", split='train[:1%]')
first_2_data = list(dataset)[:2]

for dump in first_2_data:
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  video_id = row['video_id']
  print(video_id)
  try:
    download_video(video_id)
    video_path = f"/content/videos/{video_id}.mp4"
    # !scenedetect -i {video_path} -o /content/adaptive --drop-short-scenes detect-adaptive split-video
    !scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content split-video
    # !scenedetect -i {video_path} -o /content/threshold --drop-short-scenes detect-threshold split-video
  except:
    pass

In [ ]:
%cd /content

!pip install yt_dlp scenedetect[opencv]

import json
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(f"https://www.imdb.com/video/{id}")
    return f"/content/videos/{id}.mp4"

import requests, json
url_popular= "https://camenduru-imdb.hf.space/api/chart/popular"
response_popular = requests.get(url_popular)
if response_popular.status_code == 200:
  json_data_popular = response_popular.json()
  json_string_popular = json.dumps(json_data_popular)
  movies = json.loads(json_string_popular)['data']
  # movies = movies[2:] # skip first 2
  for movie in movies:
    url_titles = f"https://camenduru-imdb.hf.space/api/titles/{movie['titleId']}"
    response_titles = requests.get(url_titles)
    if response_titles.status_code == 200:
      json_data_titles = response_titles.json()
      json_string_titles = json.dumps(json_data_titles)
      videos = json.loads(json_string_titles)['data']['videos']
      my_array = []
      my_dict = {}
      for video in videos:
        if 'Trailer' in video['duration']:
          minutes, seconds = video['duration'].split()[1].split(":")
          minutes = int(minutes) * 60
          seconds = int(seconds)
          total_seconds = minutes + seconds
          my_array.append(total_seconds)
          my_dict[total_seconds] = video['id']
      download_video_id = my_dict.get(max(my_array))
      print(download_video_id)
      download_video(download_video_id)
      video_path = f"/content/videos/{download_video_id}.mp4"
      !scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content split-video save-images -n 1
    else:
      print("Error getting JSON data:", response.status_code)

else:
  print("Error getting JSON data:", response.status_code)

In [ ]:
import os

def find_top_10_largest_files(folder_path):
    file_sizes = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            file_size = os.path.getsize(file_path)
            file_sizes.append((file_path, file_size))

    sorted_files = sorted(file_sizes, key=lambda x: x[1], reverse=True)
    top_10_files = sorted_files[:10]
    return top_10_files

folder_path = "/content/content"
top_10_files = find_top_10_largest_files(folder_path)
for file_path, file_size in top_10_files:
    print(f"File: {file_path} | Size: {file_size} bytes")

In [ ]:
%cd /content

!git clone -b dev https://github.com/camenduru/Video-BLIP2-Preprocessor
%cd /content/Video-BLIP2-Preprocessor
!pip install -r requirements.txt

In [ ]:
!python preprocess.py --video_directory /content/content --config_name "My Videos" --config_save_name "my_videos"

In [ ]:
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/test_blip2/resolve/main/test_content.zip -d /content/models -o test_content.zip
!unzip /content/models/test_content.zip

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/scheduler/scheduler_config.json -d /content/models/model_scope_diffusers/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/text_encoder/config.json -d /content/models/model_scope_diffusers/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/resolve/main/text_encoder/pytorch_model.bin -d /content/models/model_scope_diffusers/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/tokenizer/merges.txt -d /content/models/model_scope_diffusers/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/tokenizer/special_tokens_map.json -d /content/models/model_scope_diffusers/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/tokenizer/tokenizer_config.json -d /content/models/model_scope_diffusers/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/tokenizer/vocab.json -d /content/models/model_scope_diffusers/tokenizer -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/unet/config.json -d /content/models/model_scope_diffusers/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/resolve/main/unet/diffusion_pytorch_model.bin -d /content/models/model_scope_diffusers/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/vae/config.json -d /content/models/model_scope_diffusers/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/resolve/main/vae/diffusion_pytorch_model.bin -d /content/models/model_scope_diffusers/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/raw/main/model_index.json -d /content/models/model_scope_diffusers -o model_index.json

In [ ]:
!git clone https://github.com/camenduru/Text-To-Video-Finetuning
%cd Text-To-Video-Finetuning
!git clone https://huggingface.co/damo-vilab/text-to-video-ms-1.7b /content/models/model_scope_diffusers/
!pip install -r requirements.txt

In [ ]:
!python train.py --config /content/Text-To-Video-Finetuning/configs/v2/train_config.yaml

In [ ]:
pretrained_model_path: "/content/models/model_scope_diffusers/" #https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/tree/main
output_dir: "/content/outputs"
dataset_types: 
  - 'json'
offset_noise_strength: 0.1
use_offset_noise: False
extend_dataset: False
cache_latents: True
cached_latent_dir: null
train_text_encoder: False
use_unet_lora: True
use_text_lora: True
unet_lora_modules:
  - "ResnetBlock2D"
text_encoder_lora_modules:
  - "CLIPEncoderLayer"
lora_rank: 16
train_data:
  width: 512      
  height: 512
  use_bucketing: True
  sample_start_idx: 1
  fps: 24 
  frame_step: 5
  n_sample_frames: 8
  json_path: '/content/Text-To-Video-Finetuning/test_blip2/test.json'
  single_video_path: "path/to/single/video.mp4"
  single_video_prompt: ""
  fallback_prompt: ''
  path: "path/to/folder/of/videos/"
  image_dir: 'path/to/image/directory'
  single_img_prompt: ""
validation_data:
  prompt: "duck"
  sample_preview: True
  num_frames: 16
  width: 512
  height: 512
  num_inference_steps: 25
  guidance_scale: 9
learning_rate: 5e-6
adam_weight_decay: 1e-2
extra_unet_params: null
  #learning_rate: 1e-5
  #adam_weight_decay: 1e-4
extra_text_encoder_params: null
  #learning_rate: 5e-6
  #adam_weight_decay: 0.2
train_batch_size: 1
max_train_steps: 10000
checkpointing_steps: 2500
validation_steps: 100
trainable_modules:
  - "attn1"
  - "attn2"
  - 'temp_conv'
trainable_text_modules:
  - "all"
seed: 64
mixed_precision: "fp16"
use_8bit_adam: False
gradient_checkpointing: True
text_encoder_gradient_checkpointing: False
enable_xformers_memory_efficient_attention: False
enable_torch_2_attn: True
